## MRI Upload to mri.neurodata.io

This notebook demonstrates how to upload a directory of NIFTI files to a the NeuroData store server.  To use these instructions you need to have created a __"Dataset"__ that describes the shape of the images and a __"Project"__ that names a collections of images.  When you upload images, they will populate __"Channels"__ in the database.  To create a dataset and project, go to mri.neurodata.io/nd/accounts/login/.

### Directory of NIFTI files

In the following text box, give the path to the directory of the MRI files that you want to upload.  The notebook will print a list of files that it has found in that directory.

In [1]:
from ipywidgets import widgets
from IPython.display import display
niidir = widgets.Text()
display("Input directory with .nii files", niidir)

'Input directory with .nii files'

In [2]:
import os
# get the list of files to upload
files = [ os.path.abspath(niidir.value)+'/'+f for f in os.listdir( niidir.value ) ]
print("Files found in directory {}".format(niidir.value))
for f in files:
    if os.path.isfile (f):
        print(f)

Files found in directory /home/randal/nifti/randal_images/3d/
/home/randal/nifti/randal_images/3d/3d_int32_image.nii.gz
/home/randal/nifti/randal_images/3d/3d_int16_image.nii.gz
/home/randal/nifti/randal_images/3d/3d_float_image.nii.gz
/home/randal/nifti/randal_images/3d/3d_int8_image.nii.gz


### Select files

In the following text box, you will put a python regular expresssion that derives channel names from the listed files.  Use http://regex101.com to help you build an appropriate string.  Paste the file name into test string, select python regular expresssions, and build your expression.  The channel name will be derived from capturing group 1 as shown in the match information on the right side.  For example, I used the regular expression .*(3d_.*)_image.* to capture 3d_int16 and 3d_float as channels from filenames  /.../3d_int16_image.nii.gz and /.../3d_float_image.nii.gz.  You will get to review the channel names and files prior to upload.

In [3]:
inputre = widgets.Text()
display("Regular expresion to extract channel name from each file", inputre)

'Regular expresion to extract channel name from each file'

In [47]:
import re
mritodo={}
for f in files:
    m = re.match(r"{}".format(inputre.value),f) 
    if m is not None:
        print ("Upload file {} as channel {}".format(f, m.group(1)))
        mritodo[m.group(1)] = f

Upload file /home/randal/nifti/randal_images/3d/3d_int32_image.nii.gz as channel 3d_int32
Upload file /home/randal/nifti/randal_images/3d/3d_int16_image.nii.gz as channel 3d_int16
Upload file /home/randal/nifti/randal_images/3d/3d_float_image.nii.gz as channel 3d_float
Upload file /home/randal/nifti/randal_images/3d/3d_int8_image.nii.gz as channel 3d_int8


### Select a Project

Specify the project name on mri.neurodata.io that you already created.

In [50]:
proj = widgets.Text()
display("Project on mri.neurodata.io for mri upload", proj)

'Project on mri.neurodata.io for mri upload'

Widget Javascript not detected.  It may not be installed or enabled properly.


### Upload the data

Having chosen a project and verified the files and channels that you want to upload, running the following code block will send the data to the server, creating channels for each file.  If your upload fails or is interrupted, you may run this code block multiple times.  It will send the data that has not already been uploaded and omit the data that is already on the server.

In [74]:
import requests
import json

# get the current list of channels
requests.packages.urllib3.disable_warnings()
r = requests.get ( "https://mri.neurodata.io/nd/sd/{}/info/".format(proj.value), verify=False )
infos = json.loads(r.text)
chans = infos["channels"].keys()

for (k,v) in mritodo.items():
    if k in chans:
        print("Skipping already uploaded channel {}".format(k))
        continue
    
    url = 'https://mri.neurodata.io/nd/sd/{}/{}/nii/create/'.format(proj.value, k)

    print("Uploading {} to {}".format(v,url))
    requests.packages.urllib3.disable_warnings()
    response = requests.post(url, open(v, "rb").read(), verify=False)
    print("Status {}. Message {}.".format(response.status_code, response._content ))


Skipping already uploaded channel 3d_int16
Skipping already uploaded channel 3d_int8
Skipping already uploaded channel 3d_int32
Skipping already uploaded channel 3d_float
